# Object Detection Performance Analysis with YOLOv8 and Neo4j

## Overview
This notebook evaluates the performance of YOLOv8 object detection model by comparing its predictions against ground truth data. The results are stored and queried using a Neo4j graph database, allowing for efficient analysis of relationships between detected objects.

## Environment Setup
- **YOLOv8**: Using the 'yolov8x-oiv7.pt' model for object detection
- **Neo4j**: Graph database for storing and querying image-object relationships
- **Database Connection**: Using local Neo4j instance (localhost:7687)

## Data Sources
- `ground_truth_top_level.csv`: Contains ground truth annotations
- `objects_extract_smaller_set.csv`: Object detection results
- `parent_map.csv`: Hierarchical mapping between object classes
- `child_map.csv`: Child-parent relationships between object classes

## Workflow
1. **Data Loading**: Import CSVs containing ground truth and object detection data
2. **Data Preprocessing**: Convert object detection descriptions to proper format
3. **Class Hierarchy Mapping**: Map detected objects to their top-level parent classes
4. **Neo4j Queries**: Execute queries to find images containing specific objects
5. **Evaluation**: Calculate precision and recall metrics to evaluate model performance
6. **Analysis**: Compare "ALL" vs "ANY" matching strategies for object detection

## Key Functions
- `get_top_parent()`: Maps object classes to their parent classes
- `convert_to_top_level()`: Converts detailed object descriptions to top-level categories
- `query_images()`: Finds images containing ALL specified objects
- `query_images_any()`: Finds images containing ANY of the specified objects
- `find_images_in_ground_truth()`: Searches ground truth data for images with specific classes

## Results
The notebook calculates precision and recall metrics for both strict (ALL) and relaxed (ANY) matching strategies to evaluate detection performance.

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import json
import os
from neo4j import GraphDatabase
import cv2
from ultralytics import YOLO
from glob import glob
import ast

## Database Connection and Model Setup

This section initializes the key components needed for our analysis:

### Neo4j Database Connection
- **URI**: Connection to local Neo4j instance at `localhost:7687`
- **Credentials**: Using default username (`neo4j`) and configured password

### YOLOv8 Model
- Loading pre-trained YOLOv8x model with Open Images V7 weights
- This model will be used for object detection in images
- The model contains 601 classes for diverse object recognition

These components enable us to:
1. Run object detection on images
2. Store detection results in the graph database
3. Query relationships between images and detected objects

In [2]:
URI = "neo4j://localhost:7687"  
USERNAME = "neo4j"            
PASSWORD = "Tai123321"     
model = YOLO('yolov8x-oiv7.pt') 

## Initialize variables

In [3]:
ground_truth = pd.read_csv("ground_truth.csv")
object_detection = pd.read_csv("./objects_extract_fixed.csv")
parent_map = pd.read_csv("parent_map.csv")
child_map = pd.read_csv("child_map.csv")    

In [5]:
print(ground_truth.head())

            ImageID                                        Description
0  0000253ea4ecbf19  ['Carnivore', 'Human leg', 'Mammal', 'Plant', ...
1  0000286a5c6a3eb5  ['Human eye', 'Sunglasses', 'Shorts', 'Person'...
2  00003bfccf5f36c2  ['Person', 'Lantern', 'Chair', 'Table', 'Tree'...
3  000045257f66b9e2  ['Boy', 'Person', 'Cowboy hat', 'Hat', 'Fedora...
4  0000530c47410921          ['Toy', 'Bird', 'Duck', 'Tire', 'Animal']


In [6]:
object_detection["Description"] = object_detection["Description"].apply(lambda x: ast.literal_eval(x))
print(object_detection.head())

            ImageID                                        Description
0  00408efc3cabdc4e  [Food, Fast food, Waffle, Pancake, Burrito, Sn...
1  0059eb01bba96297  [Food, Fast food, Waffle, Pancake, Burrito, Sn...
2  0055444dd2ab3489                    [Person, Man, Woman, Boy, Girl]
3  0028e126ab55ebfc  [Land vehicle, Person, Furniture, Clothing, Am...
4  00336bd08f30f7f0  [Person, Human body, Clothing, Man, Woman, Boy...


In [7]:
child_map["Child"] = child_map["Child"].apply(lambda x: ast.literal_eval(x))
print(child_map.head())

        Parent                                              Child
0     Aircraft                     [Helicopter, Airplane, Rocket]
1       Animal  [Shellfish, Bird, Invertebrate, Mammal, Reptil...
2    Auto part  [Vehicle registration plate, Wheel, Seat belt,...
3  Baked goods    [Pretzel, Cookie, Muffin, Bagel, Bread, Pastry]
4         Ball  [Football, Cricket ball, Volleyball (Ball), Te...


In [8]:
ground_truth["Description"] = ground_truth["Description"].apply(lambda x: ast.literal_eval(x))
print(ground_truth.head())

            ImageID                                        Description
0  0000253ea4ecbf19  [Carnivore, Human leg, Mammal, Plant, Tree, Hu...
1  0000286a5c6a3eb5  [Human eye, Sunglasses, Shorts, Person, Human ...
2  00003bfccf5f36c2  [Person, Lantern, Chair, Table, Tree, Furnitur...
3  000045257f66b9e2  [Boy, Person, Cowboy hat, Hat, Fedora, Human b...
4  0000530c47410921                    [Toy, Bird, Duck, Tire, Animal]


In [9]:
print(ground_truth.shape)

(97273, 2)


## Neo4j get driver function

In [10]:
def get_neo4j_driver():
    return GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

## Function to query images in Neo4j containing all specified objects

In [11]:
def query_images(sorted_classes):
    image_ids = []
    with get_neo4j_driver().session() as session:
                query = """
                    WITH $classes AS classes
                    MATCH (img:Image)
                    WHERE ALL(class IN classes WHERE class IN[(img)-[:HAS_A]->(cls:Class) | cls.name]) 
                    RETURN img.image_Id AS image_id
                """
                result = session.run(query, classes=list(sorted_classes))
                for record in result:
                    if record["image_id"] not in image_ids:
                        image_ids.append(record["image_id"])
    if image_ids:
        return image_ids
    else:
        return None

### Function to query images in Neo4j containing any specified objects

In [12]:
def query_images_any(sorted_classes):
    image_ids = []
    with get_neo4j_driver().session() as session:
                query = """
                    WITH $classes AS classes
                    MATCH (img:Image)
                    WHERE ANY(class IN classes WHERE class IN[(img)-[:HAS_A]->(cls:Class) | cls.name])
                    RETURN img.image_Id AS image_id
                """
                result = session.run(query, classes=list(sorted_classes))
                for record in result:
                    if record["image_id"] not in image_ids:
                        image_ids.append(record["image_id"])
    if image_ids:
        return image_ids
    else:
        return None

### Function need fixing

In [61]:
# ! function need some modification
def find_images_in_ground_truth(sorted_classes):
    image_ids = []
    for index, row in ground_truth.iterrows():
        death_flag = False
        for class_name in sorted_classes:
            if class_name not in row["Top Level Classes"]:
                death_flag = True
                break
        if not death_flag:
            image_ids.append(row["ImageID"])
    if image_ids:
        return image_ids
    else:
        return None

### create a function to get all images that contain at least one of the queried objects

In [13]:
def get_image_in_ground_truth(classes_list):
    image_ids = []
    for index, row in ground_truth.iterrows():
        for class_name in classes_list:
            if row["Description"].count(class_name) > 0:
                image_ids.append(row["ImageID"])
                break
    if image_ids:
        return image_ids
    else:
        return None

In [32]:
pre_rec_filter = pd.read_csv("./pre_rec_single_class_any_filtered_with_fpr.csv")


In [23]:
number_of_images_per_parent = pd.DataFrame(columns=["Parent", "Number of Images"])
for index, row in child_map.iterrows():
    parent = row["Parent"]
    if parent not in pre_rec_filter["Class"].values:
        continue
    children = row["Child"]
    children.append(parent)
    
    image_ids = get_image_in_ground_truth(children)
    if image_ids is not None:
        number_of_images_per_parent.loc[len(number_of_images_per_parent)] = [parent, len(image_ids)]
        print(f"Parent: {parent}, Number of Images: {len(image_ids)}")

Parent: Aircraft, Number of Images: 829
Parent: Animal, Number of Images: 9592
Parent: Auto part, Number of Images: 5142
Parent: Baked goods, Number of Images: 835
Parent: Ball, Number of Images: 528
Parent: Bathroom accessory, Number of Images: 74
Parent: Bear, Number of Images: 162
Parent: Bed, Number of Images: 263
Parent: Beetle, Number of Images: 148
Parent: Bird, Number of Images: 2047
Parent: Boat, Number of Images: 1581
Parent: Building, Number of Images: 12404
Parent: Car, Number of Images: 5936
Parent: Carnivore, Number of Images: 2432
Parent: Clock, Number of Images: 305
Parent: Clothing, Number of Images: 36380
Parent: Container, Number of Images: 1762
Parent: Cosmetics, Number of Images: 114
Parent: Couch, Number of Images: 273
Parent: Dairy Product, Number of Images: 169
Parent: Dessert, Number of Images: 808
Parent: Door, Number of Images: 780
Parent: Drink, Number of Images: 1812
Parent: Fashion accessory, Number of Images: 3631
Parent: Fast food, Number of Images: 1098

In [27]:
number_of_images_per_parent = number_of_images_per_parent.sort_values(by="Number of Images", ascending=False)
number_of_images_per_parent.to_csv("number_of_images_per_parent.csv", index=False)
ten_most_images = number_of_images_per_parent.head(10)


          Parent  Number of Images
50        Person             46552
15      Clothing             36380
52         Plant             25566
34    Human body             21700
69          Tree             17804
73       Vehicle             13232
11      Building             12404
40  Land vehicle             10122
1         Animal              9592
28      Footwear              7648
Amount of Images: 201000


In [40]:
for index, row in number_of_images_per_parent.iterrows():
    parent = row["Parent"]
    if parent not in pre_rec_filter["Class"].values:
        number_of_images_per_parent = number_of_images_per_parent.drop(index)
        continue

for index, row in number_of_images_per_parent.iterrows():
    if row["Parent"] == "Building":
        number_of_images_per_parent = number_of_images_per_parent.drop(index)
    elif row["Parent"] == "Furniture":
        number_of_images_per_parent = number_of_images_per_parent.drop(index)
    elif row["Parent"] == "Food":
        number_of_images_per_parent = number_of_images_per_parent.drop(index)
        
print(number_of_images_per_parent.head(9))
classes_to_query = number_of_images_per_parent["Parent"].head(9).values
print(classes_to_query)

          Parent  Number of Images
50        Person             46552
15      Clothing             36380
52         Plant             25566
34    Human body             21700
69          Tree             17804
40  Land vehicle             10122
28      Footwear              7648
12           Car              5936
2      Auto part              5142
['Person' 'Clothing' 'Plant' 'Human body' 'Tree' 'Land vehicle' 'Footwear' 'Car' 'Auto part']


In [41]:
actual_ground_truth = []  
for index, row in child_map.iterrows():
    parent = row["Parent"]
    if parent in classes_to_query:
        children = row["Child"]
        children.append(parent)
        all_images = get_image_in_ground_truth(children)
        if all_images:
            for image in all_images:
                if image not in actual_ground_truth:
                    actual_ground_truth.append(image)
    
        print(len(actual_ground_truth))
        print("-"*50)

print(len(actual_ground_truth))

5142
--------------------------------------------------
8455
--------------------------------------------------
43196
--------------------------------------------------
43226
--------------------------------------------------
47713
--------------------------------------------------
49269
--------------------------------------------------
56473
--------------------------------------------------
73749
--------------------------------------------------
73833
--------------------------------------------------
73833


In [ ]:
def get_image_in_ground_truth_small(classes_list):
    image_ids = []
    for index, row in object_detection.iterrows():
        for class_name in classes_list:
            if row["Description"].count(class_name) > 0:
                image_ids.append(row["ImageID"])
                break
    if image_ids:
        return image_ids
    else:
        return None

In [14]:
def is_image_in_ground_truth_small(image_id):
    for index, row in object_detection.iterrows():
        if row["ImageID"] == image_id:
            return True
    return False

In [42]:
# list_precision = []
# list_recall = []
list_precision_any = []
list_recall_any = []
list_false_positive_rate = []
pre_rec_single_class_pd = pd.DataFrame(columns=["Class", "Precision", "Recall", "False Positive Rate"])
ground_truth_len = len(actual_ground_truth)

for index, row in child_map.iterrows():
    parent_class = row["Parent"]

    if parent_class not in classes_to_query:
        continue
    
    child_class = row["Child"]
    child_class.append(row["Parent"])
    
    child_class = sorted(child_class)
    
    all_images = get_image_in_ground_truth(child_class)
    
    output_images_any = query_images_any(child_class)

    if all_images is None or output_images_any is None:
        continue

    if len(all_images) == 0 or len(output_images_any) == 0:
        continue

    actual_negative = ground_truth_len - len(all_images)
    true_positive = 0
    false_positive = 0

    for image_id in output_images_any:
        if image_id in all_images:
            true_positive += 1
        else:
            false_positive += 1

    true_negative = actual_negative - false_positive
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / len(all_images)
    fpr = false_positive / actual_negative

    print("Class: ", parent_class)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("False Positive Rate: ", fpr)
    print("True Positive: ", true_positive)
    print("False Positive: ", false_positive)
    print("Total: ", len(all_images))
    print("Output: ", len(output_images_any))
    print("_"*50)

    list_precision_any.append(precision)
    list_recall_any.append(recall)
    list_false_positive_rate.append(fpr)
    pre_rec_single_class_pd.loc[len(pre_rec_single_class_pd)] = [parent_class, precision, recall, fpr]

# print("Average Precision: ", np.mean(list_precision))
# print("Average Recall: ", np.mean(list_recall))

print("Average Precision Any: ", np.mean(list_precision_any))
print("Average Recall Any: ", np.mean(list_recall_any))
print("Average False Positive Rate: ", np.mean(list_false_positive_rate))
## add average precision and recall to pre_rec_single_class_pd
# pre_rec_single_class_pd.loc[len(pre_rec_single_class_pd)] = ["Average", np.mean(list_precision_any), np.mean(list_recall_any), np.mean(list_false_positive_rate)]
# pre_rec_single_class_pd.to_csv("pre_rec_single_class_any_with_fpr.csv", index=False)

Class:  Auto part
Precision:  0.6525511975008678
Recall:  0.7312329832749903
False Positive Rate:  0.02914501171914807
True Positive:  3760
False Positive:  2002
Total:  5142
Output:  5762
__________________________________________________
Class:  Car
Precision:  0.8087472201630838
Recall:  0.9189690026954178
False Positive Rate:  0.018999366687777075
True Positive:  5455
False Positive:  1290
Total:  5936
Output:  6745
__________________________________________________
Class:  Clothing
Precision:  0.8410331438861066
Recall:  0.8216602528862013
False Positive Rate:  0.15085573919312203
True Positive:  29892
False Positive:  5650
Total:  36380
Output:  35542
__________________________________________________
Class:  Footwear
Precision:  0.6752136752136753
Recall:  0.7333943514644351
False Positive Rate:  0.040764523683614114
True Positive:  5609
False Positive:  2698
Total:  7648
Output:  8307
__________________________________________________
Class:  Human body
Precision:  0.7658016682

In [42]:
pre_rec_single_class_pd.shape

(77, 3)

In [51]:
precision_recall_single_class = pd.read_csv("pre_rec_single_class_any.csv")
print(precision_recall_single_class.head())
print(precision_recall_single_class.shape)

         Class  Precision    Recall
0     Aircraft   0.877622  0.908323
1       Animal   0.901511  0.192765
2    Auto part   0.652551  0.731233
3  Baked goods   0.809035  0.471856
4         Ball   0.737475  0.696970
(77, 3)


In [52]:
# drop row that has pre or recall less than 0.5
precision_recall_single_class = precision_recall_single_class.drop(precision_recall_single_class[(precision_recall_single_class["Precision"] < 0.5) | (precision_recall_single_class["Recall"] < 0.5)].index)

In [53]:
print(precision_recall_single_class.tail())
print(precision_recall_single_class.shape)

         Class  Precision    Recall
71      Turtle   0.781250  0.833333
72   Vegetable   0.799205  0.562238
74  Watercraft   0.824324  0.853644
75      Weapon   0.713528  0.652913
76     Average   0.744093  0.619837
(57, 3)


In [54]:
precision_recall_single_class = precision_recall_single_class.drop(precision_recall_single_class[precision_recall_single_class["Class"] == "Average"].index)
print(precision_recall_single_class.shape)
print(precision_recall_single_class.tail())

(56, 3)
         Class  Precision    Recall
70    Trousers   0.526421  0.624406
71      Turtle   0.781250  0.833333
72   Vegetable   0.799205  0.562238
74  Watercraft   0.824324  0.853644
75      Weapon   0.713528  0.652913


In [59]:
copy_precision_recall_single_class = precision_recall_single_class.copy()
average_precision = np.mean(copy_precision_recall_single_class["Precision"])
average_recall = np.mean(copy_precision_recall_single_class["Recall"])
print("Average Precision: ", average_precision)
print("Average Recall: ", average_recall)
copy_precision_recall_single_class.loc[len(copy_precision_recall_single_class)] = ["Average", average_precision, average_recall]
print(copy_precision_recall_single_class.tail())
copy_precision_recall_single_class.to_csv("pre_rec_single_class_any_filtered.csv", index=False)

Average Precision:  0.755617985855299
Average Recall:  0.7292268225094621
         Class  Precision    Recall
70    Trousers   0.526421  0.624406
71      Turtle   0.781250  0.833333
72   Vegetable   0.799205  0.562238
74  Watercraft   0.824324  0.853644
75      Weapon   0.713528  0.652913


### Calculate True Positives Rate (TPR) and False Positives Rate (FPR) into new df

In [61]:
precision_recall_fpr = pd.read_csv("pre_rec_single_class_any_with_fpr.csv")
print(precision_recall_fpr.head())

         Class  Precision    Recall  False Positive Rate
0     Aircraft   0.877622  0.908323             0.001089
1       Animal   0.901511  0.192765             0.002304
2    Auto part   0.652551  0.731233             0.021730
3  Baked goods   0.809035  0.471856             0.000964
4         Ball   0.737475  0.696970             0.001354


In [63]:
precision_recall_fpr = precision_recall_fpr.drop(precision_recall_fpr[(precision_recall_fpr["Precision"] < 0.5) | (precision_recall_fpr["Recall"] < 0.5)].index)
print(precision_recall_fpr.shape)
print(precision_recall_fpr.tail())

(57, 4)
         Class  Precision    Recall  False Positive Rate
71      Turtle   0.781250  0.833333             0.000216
72   Vegetable   0.799205  0.562238             0.001046
74  Watercraft   0.824324  0.853644             0.003265
75      Weapon   0.713528  0.652913             0.001115
76     Average   0.744093  0.619837             0.008089


In [64]:
precision_recall_fpr = precision_recall_fpr.drop(precision_recall_fpr[precision_recall_fpr["Class"] == "Average"].index)
precision_recall_fpr.reindex()
print(precision_recall_fpr.shape)
print(precision_recall_fpr.tail())

(56, 4)
         Class  Precision    Recall  False Positive Rate
70    Trousers   0.526421  0.624406             0.012431
71      Turtle   0.781250  0.833333             0.000216
72   Vegetable   0.799205  0.562238             0.001046
74  Watercraft   0.824324  0.853644             0.003265
75      Weapon   0.713528  0.652913             0.001115


In [65]:
precision_recall_fpr.loc[len(precision_recall_fpr)] = ["Average", np.mean(precision_recall_fpr["Precision"]), np.mean(precision_recall_fpr["Recall"]), np.mean(precision_recall_fpr["False Positive Rate"])]
print(precision_recall_fpr.tail())

         Class  Precision    Recall  False Positive Rate
70    Trousers   0.526421  0.624406             0.012431
71      Turtle   0.781250  0.833333             0.000216
72   Vegetable   0.799205  0.562238             0.001046
74  Watercraft   0.824324  0.853644             0.003265
75      Weapon   0.713528  0.652913             0.001115


In [66]:
precision_recall_fpr.to_csv("pre_rec_single_class_any_filtered_with_fpr.csv", index=False)

In [3]:
pre_rec = pd.read_csv("precision_recall_single_class.csv")

for index, row in pre_rec.iterrows():
    if row["Precision"] == 0 and row["Recall"] == 0:
        pre_rec.drop(index, inplace=True)

print("Average Precision: ", pre_rec["Precision"].mean())
print("Average Recall: ", pre_rec["Recall"].mean())

Average Precision:  0.662120942963803
Average Recall:  0.5520499073785102
